In [7]:
import cv2


In [8]:
import numpy as np

In [9]:
import pytesseract

In [10]:
pytesseract.pytesseract.tesseract_cmd=r"c:\Program Files\Tesseract-OCR\tesseract.exe"

In [11]:
num_plate_cascade=cv2.CascadeClassifier(r"D:\DL projects\car_numberplate\haarcascade_russian_plate_number.xml")
num_plate_cascade

< cv2.CascadeClassifier 00000283B870E7B0>

In [12]:
def extract_numbers(img_filename):
    img=cv2.imread(img_filename)
    inverted_img=cv2.bitwise_not(img)
    cv2.imwrite("temp\inverted.jpg",inverted_img)
    img_gray=cv2.cvtColor(inverted_img,cv2.COLOR_RGB2GRAY)
    thresh,binary_img=cv2.threshold(img_gray,3,21,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    conv_img=cv2.cvtColor(inverted_img,cv2.COLOR_BGR2RGB)
    nplate=num_plate_cascade.detectMultiScale(conv_img,scaleFactor=1.1,minNeighbors=4)
    #width height and channel
    for (x,y,w,h) in nplate:
        a,b=(int(0.025*img.shape[0]),int(0.025*img.shape[1]))
        plate=img[y+a:y+h-a,x+b:x+w-b,:]
        #kernel is a black window
        kernel=np.ones((1,1),np.uint8)
        plate=cv2.dilate(plate,kernel,iterations=1)
        plate=cv2.erode(plate,kernel,iterations=1)
        gray_plate=cv2.cvtColor(plate,cv2.COLOR_BGR2GRAY)
        (thresh,plate)=cv2.threshold(gray_plate,127,255,cv2.THRESH_BINARY)
        #read the text on the plate
        read=pytesseract.image_to_string(plate)
        read= ''.join(e for e in read if e.isalnum())
        stat=read[0:2]
        cv2.rectangle(img,(x,y),(x+w,y+h),(51,51,255),2)
        cv2.rectangle(img,(x,y-40),(x+w,y),(51,51,255),-1)
        cv2.putText(img,read,(x,y-10),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=0.7,color=(255,255,255),thickness=2)
        

        cv2.imshow("plate",plate)
        




    cv2.imshow('Result',img)
    cv2.waitKey()
    cv2.destroyAllWindows()

extract_numbers("carplate.jpg")